In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.module
import strawb.tools

import plotly.express as px
import pandas
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

In [ ]:
# in case execute db.load_entire_db_from_ONC() to load the entire db
db = strawb.SyncDBHandler()  # loads the db
db.load_onc_db_update(output=True, save_db=True)

## Select the files of interest

In [ ]:
timestamp = np.datetime64('2021-09-01T00:00')  # the time of interst
timerange = np.timedelta64(7*24, "h")  # select files covering the [timestamp, timestamp+timerange]

# mask by device
mask = db.dataframe.dataProductCode == 'SMRD'
mask &= db.dataframe.deviceCode == 'TUMPMTSPECTROMETER001'

# and add mask for synced files only
# mask &= db.dataframe.synced

## select 1h frame around one bioluminescence event
# timestamp = np.datetime64('2021-09-04T23:44:09')
time_from = np.datetime64(timestamp, "h")  # round down to full hour
time_to = np.datetime64(timestamp + timerange, "h")  # round down to full hour

# files which cover time_from_mask
mask_from = db.dataframe.dateFrom <= pandas.Timestamp(time_from, tz="UTC")
mask_from &= db.dataframe.dateTo >= pandas.Timestamp(time_from, tz="UTC")
# files which cover time_to_mask
mask_to = db.dataframe.dateFrom <= pandas.Timestamp(time_to, tz="UTC")
mask_to &= db.dataframe.dateTo >= pandas.Timestamp(time_to, tz="UTC")

# files which cover time_to_mask
mask_to = db.dataframe.dateFrom >= pandas.Timestamp(time_from, tz="UTC")
mask_to &= db.dataframe.dateTo <= pandas.Timestamp(time_to, tz="UTC")

mask &= mask_from | mask_to

### show selected file from the DB - here its 1 file only
db.dataframe[mask]

### Donwload files, if not all are synced

In [ ]:
### this will download the selected file
if not db.dataframe.synced[mask].all():
    db.update_db_and_load_files(
        db.dataframe[mask],
        output=True,  # print output to console
        download=True,  # download the files
        #save_db=True
    )
    db.save_db()

### Load the file
Handles also multiple files in the timerange. Creats an virtualHDF5 and load it.

In [ ]:
# select the Module file(s) -> dataProductCode == 'MSSCD'
item = db.dataframe[mask & (db.dataframe.dataProductCode == 'SMRD')]

try:  # if the camera file is still open
    module.file_handler.close()
except:
    pass
    
# generate a virtual hdf5 to combine the datasets if there are multiple files selected
if len(item) > 1:
    vhdf5 = strawb.VirtualHDF5('SMRD_event_view.hdf5', item.fullPath.to_list())  
    file_name = vhdf5.file_name
else:
    file_name = item.fullPath[0]
    
module = strawb.sensors.Module(file=file_name)
module.file_handler.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# Plotting with pandas and plotly

In [ ]:
df_accel = pandas.DataFrame(dict(time = module.file_handler.accel_time.asdatetime()[:],
                                 x    = module.file_handler.accel_x,
                                 y    = module.file_handler.accel_y,
                                 z    = module.file_handler.accel_z,))

In [ ]:
df_magneto = pandas.DataFrame(dict(time = module.file_handler.magneto_time.asdatetime()[:],
                                   x    = module.file_handler.magneto_x,
                                   y    = module.file_handler.magneto_y,
                                   z    = module.file_handler.magneto_z,))

In [ ]:
df_pth = pandas.DataFrame(dict(time        = module.file_handler.pth_time.asdatetime()[:],
                               humidity    = module.file_handler.pth_humidity,
                               temperature = module.file_handler.pth_temperature,
                               pressure    = module.file_handler.pth_pressure,))

In [ ]:
df_temperatures = pandas.DataFrame(dict(time  = module.file_handler.temperatures_time.asdatetime()[:],
                                        temp1 = module.file_handler.temperatures_temp1,
                                        temp2 = module.file_handler.temperatures_temp2,
                                        temp3 = module.file_handler.temperatures_temp3))

In [ ]:
df_pwrmoni = pandas.DataFrame(dict(time           = module.file_handler.pwrmoni_time.asdatetime()[:],
                                   c2_current     = module.file_handler.pwrmoni_c2_current,
                                   c2_voltage     = module.file_handler.pwrmoni_c2_voltage,
                                   laser_current  = module.file_handler.pwrmoni_laser_current,
                                   laser_voltage  = module.file_handler.pwrmoni_laser_voltage,
                                   motor_current  = module.file_handler.pwrmoni_motor_current,
                                   motor_voltage  = module.file_handler.pwrmoni_motor_voltage,
                                   padiwa_current = module.file_handler.pwrmoni_padiwa_current,
                                   padiwa_voltage = module.file_handler.pwrmoni_padiwa_voltage,
                                   switch_current = module.file_handler.pwrmoni_switch_current,
                                   switch_voltage = module.file_handler.pwrmoni_switch_voltage,
                                   trb3sc_current = module.file_handler.pwrmoni_trb3sc_current,
                                   trb3sc_voltage = module.file_handler.pwrmoni_trb3sc_voltage))

In [ ]:
px.line(df_accel, x='time', y=['x', 'y', 'z'])

In [ ]:
px.line(df_magneto, x='time', y=['x', 'y', 'z'])

In [ ]:
px.line(df_pth, x='time', y=['humidity','temperature','pressure'])

In [ ]:
px.line(df_temperatures, x='time', y=['temp1','temp2','temp3'])

In [ ]:
px.line(df_pwrmoni, x='time', y=['c2_current', 'c2_voltage',
                                 'laser_current', 'laser_voltage',
                                 'motor_current', 'motor_voltage',
                                 'padiwa_current', 'padiwa_voltage',
                                 'switch_current', 'switch_voltage',
                                 'trb3sc_current', 'trb3sc_voltage'])

# Load a file
## can be a single or combined aka. virtual HDF5 file

In [ ]:
fig, ax = plt.subplots(nrows=8, squeeze=False, figsize=(7,15), sharex=True)
ax = ax.flatten()

i = 0

# ACCEL
strawb.tools.plot_binned_mean(module.file_handler.accel_time, 
                              module.file_handler.accel_x[:]*9.81, ax=ax[i], label='accel x')
strawb.tools.plot_binned_mean(module.file_handler.accel_time, 
                              module.file_handler.accel_y[:]*9.81, ax=ax[i], label='accel y')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1
strawb.tools.plot_binned_mean(module.file_handler.accel_time, 
                              module.file_handler.accel_z[:]*9.81, ax=ax[i], label='accel z')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1

# Magnet
strawb.tools.plot_binned_mean(module.file_handler.magneto_time, 
                              module.file_handler.magneto_x, ax=ax[i], label='magneto x')
strawb.tools.plot_binned_mean(module.file_handler.magneto_time, 
                              module.file_handler.magneto_y, ax=ax[i], label='magneto y')
strawb.tools.plot_binned_mean(module.file_handler.magneto_time, 
                              module.file_handler.magneto_z, ax=ax[i], label='magneto z')
ax[i].set_ylabel('Magneto [a.u.]')
i += 1

# TEMP.
strawb.tools.plot_binned_mean(module.file_handler.temperatures_time,
                              module.file_handler.temperatures_temp1, ax=ax[i], label='temp 1')
strawb.tools.plot_binned_mean(module.file_handler.temperatures_time,
                              module.file_handler.temperatures_temp2, ax=ax[i], label='temp 2')
ax[i].set_ylabel('Temp [C]')
i += 1
strawb.tools.plot_binned_mean(module.file_handler.temperatures_time, 
                              module.file_handler.temperatures_temp3, ax=ax[i], label='temp 3')
ax[i].set_ylabel('Temp [C]')
i += 1

# PTH
strawb.tools.plot_binned_mean(module.file_handler.pth_time, 
                              module.file_handler.pth_temperature, ax=ax[i], label='temp pth')
ax[i].set_ylabel('Temp [C]')
i += 1

strawb.tools.plot_binned_mean(module.file_handler.pth_time, 
                              module.file_handler.pth_humidity, ax=ax[i], label='pth humidity')
ax[i].set_ylabel('humidity [per]')
i += 1

strawb.tools.plot_binned_mean(module.file_handler.pth_time, 
                              module.file_handler.pth_pressure, ax=ax[i], label='pth pressure')
ax[i].set_ylabel('pressure [mbar]')
i += 1

for ax_i in ax:
    ax_i.legend(loc=1)
    ax_i.grid()

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
fig.autofmt_xdate()
locator = mdates.AutoDateLocator()
#fmt = mdates.AutoDateFormatter(locator, usetex=True)
fmt = mdates.ConciseDateFormatter(locator)
ax[-1].xaxis.set_major_formatter(fmt)
plt.tight_layout()

## plot multiple modules (currently doesn't work: needs viratul hdf5 files or a file per moduel)

In [ ]:
# module_list = []

# for dev_i in strawb.dev_codes_deployed:
#     # get a list with all filenames that should be included in the virtual HDF5 file
#     f_names = glob.glob(f'{strawb.Config.raw_data_dir}/{dev_i}_*MODULE.hdf5')
#     f_names.sort()  # f_names should be sorted

#     # create the virtual HDF5 file, f_names should be sorted, otherwise the data isn't sorted
#     virtual_hdf5_gen = strawb.VirtualHDF5(f'{dev_i}_MODULE.h5',
#                                           f_names,
#                                           obj_dict_filter=strawb.DatasetsInGroupSameSize)

#     module_list.append(strawb.sensors.module.FileHandler(f'{dev_i}_MODULE.h5'))

In [ ]:
# fig, ax = plt.subplots(nrows=6, squeeze=False, figsize=(10,15), sharex=True)
# ax = ax.flatten()


# for module in module_list:
#     print(module.module)
#     i = 0
#     # TEMP.
#     mask = module.temperatures_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
#     plot_time = module.temperatures_time[:][mask]
#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp1[:][mask],
#                                   ax=ax[i], label=f'temp 1 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp2[:][mask],
#                                   ax=ax[i], label=f'temp 2 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp3[:][mask],
#                                   ax=ax[i], label=f'temp 3 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     # PTH
#     mask = module.pth_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
#     plot_time = module.pth_time[:][mask]
#     strawb.tools.plot_binned_mean(plot_time, module.pth_temperature[:][mask],
#                                   ax=ax[i], label=f'temp pth {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.pth_humidity[:][mask],
#                                   ax=ax[i], label=f'pth humidity {module.module}')
#     ax[i].set_ylabel('humidity [per]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.pth_pressure[:][mask],
#                                   ax=ax[i], label=f'pth pressure {module.module}')
#     ax[i].set_ylabel('pressure [mbar]')
#     i += 1

# for ax_i in ax:
#     ax_i.legend(loc=0)
#     ax_i.grid()

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
# fig.autofmt_xdate()
# locator = mdates.AutoDateLocator()
# #fmt = mdates.AutoDateFormatter(locator, usetex=True)
# fmt = mdates.ConciseDateFormatter(locator)
# ax[-1].xaxis.set_major_formatter(fmt)
# plt.tight_layout()